In [9]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
from nltk.tokenize import sent_tokenize
import numpy as np
import re
import pandas as pd
import string
from string import digits
from sklearn.utils import shuffle


## 2) Loading Preprocessed data

In [10]:
from pathlib import Path
data_file = Path("/home/iftekhar/AI-system/Mixed/POL_workshop/processed_texts.txt")
with open(data_file, encoding='utf-8') as f:
    data_list = f.read().splitlines()

In [ ]:
data_list[:1]

In [11]:
all_text = " ".join(data_list)


In [6]:
vocab = set(all_text.split())

In [7]:
len(vocab)

5465

In [8]:
vocab = list(vocab)
vocab[:5]

['多数', '駆除', 'cs', 'エクスプロラツール', '表徴']

In [12]:
all_text = " ".join(data_list)
all_text = single_character_remover(all_text)

# lines = re.sub(r"\s+", " ", lines)

# sentences = sent_tokenize(lines)
# sent = np.asarray(sentences)
#sentTwo = sent.reshape(len(sent)//2,2)

NameError: name 'single_character_remover' is not defined

In [14]:
def page_text_split(page_text, word_limit):
    page_text = page_text.split()
    chunks = [' '.join(page_text[i:i + word_limit]) for i in range(0,
                                                                   len(page_text), word_limit)]
    return chunks


def single_character_remover(text):
    collector = []
    for items in text.split():
        if len(items) < 2:
            replaced = re.sub(r'[ぁ-んァ-ン]', '', items)
            replaced = re.sub(r'[A-Za-z]', '', replaced)
            replaced = re.sub(r'[0-9]', '', replaced)
            collector.append(replaced)
        else:
            collector.append(items)

    return ' '.join([temp.strip(' ') for temp in collector])

In [48]:
lines = page_text_split(all_text, 20)

In [49]:
lines[:2]

['mobicontrol v manual mobicontrol v manual カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定 mobicontrol 設定 順序 mobicontrol 設定 順序 mobicontrol 設定',
 '順序 端末 os 対象 オンプレミス mobicontrol サーバ 必要 設定 事項 ios 端末 macos コンピュータ 必須 接続 プロファイル 作成 adds active']

In [50]:
collector = []
for line in lines:
    collector.append(line.split())

In [51]:
collector[:2]

[['mobicontrol',
  'v',
  'manual',
  'mobicontrol',
  'v',
  'manual',
  'カスタマイザー',
  'タイトル',
  '設定',
  'カスタマイザー',
  'サブタイトル',
  '設定',
  'mobicontrol',
  '設定',
  '順序',
  'mobicontrol',
  '設定',
  '順序',
  'mobicontrol',
  '設定'],
 ['順序',
  '端末',
  'os',
  '対象',
  'オンプレミス',
  'mobicontrol',
  'サーバ',
  '必要',
  '設定',
  '事項',
  'ios',
  '端末',
  'macos',
  'コンピュータ',
  '必須',
  '接続',
  'プロファイル',
  '作成',
  'adds',
  'active']]

In [34]:
sent = np.asarray(lines)

if len(sent)%2==0:
    sentTwo = sent.reshape(len(sent)//2,2)
else:
    sent = sent[:-1]
    sentTwo = sent.reshape(len(sent)//2,2)

In [58]:
sentTwo[:2]

array([['mobicontrol manual mobicontrol manual カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定',
        'mobicontrol 設定 順序 mobicontrol 設定 順序 mobicontrol 設定 順序 端末'],
       ['os 対象 オンプレミス mobicontrol サーバ 必要 設定 事項 ios 端末',
        'macos コンピュータ 必須 接続 プロファイル 作成 adds active directory domain']],
      dtype='<U134')

In [36]:

len(sentTwo)

9766

In [37]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [59]:
create_dataset(sentTwo, 2)

(array([['mobicontrol manual mobicontrol manual カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定',
         'os 対象 オンプレミス mobicontrol サーバ 必要 設定 事項 ios 端末'],
        ['os 対象 オンプレミス mobicontrol サーバ 必要 設定 事項 ios 端末',
         'service サーバ 間 ad f・s active directory federation service 間'],
        ['service サーバ 間 ad f・s active directory federation service 間',
         '端末 ios android 複数 従業員 共用 必須 apple google microsoft'],
        ...,
        ['コンソール 端末 管理 端末 ユーザ 当該 端末 デバイス pc タブレット',
         'mobicontrol 機能 管理対象 端末 端末 ユーザ 自ら 端末 限定 端末'],
        ['mobicontrol 機能 管理対象 端末 端末 ユーザ 自ら 端末 限定 端末',
         'セルフサービスポータル ログイン microsoft exchange アカウントid パスワード 従業員 私物 端末 pc'],
        ['セルフサービスポータル ログイン microsoft exchange アカウントid パスワード 従業員 私物 端末 pc',
         'サーバ フロント サーバ exchange active sync サーバ サーバ 設定 スマホ']],
       dtype='<U134'),
 array(['service サーバ 間 ad f・s active directory federation service 間',
        '端末 ios android 複数 従業員 共用 必須 apple google microsoft',
        '設定 app ストア アプリ 購入 managed google play アプリ ア

In [43]:
from pandas import DataFrame, concat
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	df = DataFrame(data)
	cols = []
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i+1))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i-1))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [44]:
question = []
answer = []
for items in collector:
    print(items)
    shifted_df = (series_to_supervised(items, 8, 8))
    for i in range(len(shifted_df)):
        saver = shifted_df.iloc[i].values.tolist()
        length = len(saver)//2
        question.append(saver[:length])
        answer.append(saver[length:]) 
    break

collect_question = []
for items in question:
    collect_question.append(' '.join(items))
collect_answer = []
for items in answer:
    collect_answer.append(' '.join(items))    

['mobicontrol', 'v', 'manual', 'mobicontrol', 'v', 'manual', 'カスタマイザー', 'タイトル', '設定', 'カスタマイザー', 'サブタイトル', '設定', 'mobicontrol', '設定', '順序', 'mobicontrol', '設定', '順序', 'mobicontrol', '設定', '順序', '端末', 'os', '対象', 'オンプレミス', 'mobicontrol', 'サーバ', '必要', '設定', '事項']


In [45]:
collect_question[:5]

['mobicontrol v manual mobicontrol v manual カスタマイザー タイトル',
 'v manual mobicontrol v manual カスタマイザー タイトル 設定',
 'manual mobicontrol v manual カスタマイザー タイトル 設定 カスタマイザー',
 'mobicontrol v manual カスタマイザー タイトル 設定 カスタマイザー サブタイトル',
 'v manual カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定']

In [46]:
collect_answer[:5]

['サブタイトル 設定 mobicontrol 設定 順序 mobicontrol 設定 順序',
 '設定 mobicontrol 設定 順序 mobicontrol 設定 順序 mobicontrol',
 'mobicontrol 設定 順序 mobicontrol 設定 順序 mobicontrol 設定',
 '設定 順序 mobicontrol 設定 順序 mobicontrol 設定 順序',
 '順序 mobicontrol 設定 順序 mobicontrol 設定 順序 端末']

In [47]:
lines = pd.DataFrame(zip(collect_question, collect_answer), columns=['Query', 'Prediction'])
lines.head()

,Query,Prediction
0,mobicontrol v manual mobicontrol v manual カスタマ...,サブタイトル 設定 mobicontrol 設定 順序 mobicontrol 設定 順序
1,v manual mobicontrol v manual カスタマイザー タイトル 設定,設定 mobicontrol 設定 順序 mobicontrol 設定 順序 mobicon...
2,manual mobicontrol v manual カスタマイザー タイトル 設定 カス...,mobicontrol 設定 順序 mobicontrol 設定 順序 mobicontro...
3,mobicontrol v manual カスタマイザー タイトル 設定 カスタマイザー サ...,設定 順序 mobicontrol 設定 順序 mobicontrol 設定 順序
4,v manual カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定,順序 mobicontrol 設定 順序 mobicontrol 設定 順序 端末


In [84]:
len(lines)

58594

In [30]:
col1, col2 = create_dataset(sentTwo)
#col1 = col1.reshape(-1)
col2 = col2.reshape(-1,1)
#col2.shape
dt = np.concatenate((col1,col2), axis=1)

lines = pd.DataFrame(dt, columns=['Query', 'Prediction'])
lines.head()

In [85]:
# Add start and end tokens to target sequences
lines.Prediction = lines.Prediction.apply(lambda x : 'START_ '+ x + ' _END')
lines.head(10)

,Query,Prediction
0,mobicontrol manual mobicontrol manual,START_ カスタマイザー タイトル 設定 カスタマイザー _END
1,manual mobicontrol manual カスタマイザー,START_ タイトル 設定 カスタマイザー サブタイトル _END
2,mobicontrol manual カスタマイザー タイトル,START_ 設定 カスタマイザー サブタイトル 設定 _END
3,mobicontrol 設定 順序 mobicontrol,START_ 設定 順序 mobicontrol 設定 _END
4,設定 順序 mobicontrol 設定,START_ 順序 mobicontrol 設定 順序 _END
5,順序 mobicontrol 設定 順序,START_ mobicontrol 設定 順序 端末 _END
6,os 対象 オンプレミス mobicontrol,START_ サーバ 必要 設定 事項 _END
7,対象 オンプレミス mobicontrol サーバ,START_ 必要 設定 事項 ios _END
8,オンプレミス mobicontrol サーバ 必要,START_ 設定 事項 ios 端末 _END
9,macos コンピュータ 必須 接続,START_ プロファイル 作成 adds active _END


In [86]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( lines.Query + lines.Prediction )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 9227


### B) Reading the data from the files

We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them.
*   Remove unwanted data types which are produced while parsing the data.
*   Append `<START>` and `<END>` to all the `answers`.
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` ) into it.






### C) Preparing data for Seq2Seq model

Our model requires three arrays namely `encoder_input_data`, `decoder_input_data` and `decoder_output_data`.

For `encoder_input_data` :
* Tokenize the `questions`. Pad them to their maximum lquestionsth.

For `decoder_input_data` :
* Tokenize the `answers`. Pad them to their maximum lquestionsth.

For `decoder_output_data` :

* Tokenize the `answers`. Remove the first element from all the `tokenized_answers`. This is the `<START>` element which we added earlier.



In [87]:
questions = lines.Query 
answers = lines.Prediction

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max([ len(x) for x in tokenized_answers ])
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
#np.save( 'Saved Arrays/enc_in_data.npy' , encoder_input_data )
#np.save( 'Saved Arrays/dec_in_data.npy' , decoder_input_data )
#np.save( 'Saved Arrays/dec_tar_data.npy' , decoder_output_data )


(58594, 6) 6
(58594, 8) 8
(58594, 8, 9227)


## 3) Defining the Encoder-Decoder model
The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors. **( Note :  Don't forget the `mask_zero=True` argument here )**
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce seqeunces.

**Important points :**


*   `200` is the output of the GloVe embeddings.
*   `embedding_matrix` is the GloVe embedding which we downloaded earlier.


<center><img style="float: center;" src="https://cdn-images-1.medium.com/max/1600/1*bnRvZDDapHF8Gk8soACtCQ.gif"></center>


Image credits to [Hackernoon](https://hackernoon.com/tutorial-3-what-is-seq2seq-for-text-summarization-and-why-68ebaa644db0).










In [37]:
import tensorflow as tf
shape = 100

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, shape , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( shape , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, shape , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( shape , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    622900      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    622900      input_2[0][0]                    
______________________________________________________________________________________________

## 4) Training the model
We train the model for a number of epochs with `RMSprop` optimizer and `categorical_crossentropy` loss function.

In [38]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=5 ) 
model.save( 'model.h5' ) 


Epoch 1/5
996/996 [==============================] - 26s 26ms/step - loss: 4.6889
Epoch 2/5
996/996 [==============================] - 25s 26ms/step - loss: 4.4408
Epoch 3/5
996/996 [==============================] - 26s 26ms/step - loss: 4.3236
Epoch 4/5
996/996 [==============================] - 25s 25ms/step - loss: 4.2058
Epoch 5/5
996/996 [==============================] - 26s 26ms/step - loss: 4.0936


## 5) Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [39]:
shape = 100

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( shape ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( shape ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


## 6) Talking with our Chatbot

First, we define a method `str_to_tokens` which converts `str` questions to Integer tokens with padding.


In [40]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    #return tokens_list
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [46]:
str_to_tokens("会社 支給 端末 ios android")

array([[138, 512,   1,  67,  24,   0,   0,   0,   0,   0,   0,   0]],
      dtype=int32)




1.   First, we take a question as input and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum answer lquestionsth.







In [48]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : 会社 支給 端末 ios android
 端末 グループ 端末 グループ 端末 グループ 端末 グループ 端末 end
Enter question : mobicontrol android setting
 端末 ユーザ ユーザ ユーザ パスワード 入力 入力 端末 end


KeyboardInterrupt: 

In [3]:
import numpy as np
import pickle
from tensorflow.keras import layers, activations, models, preprocessing, utils, optimizers, activations 
from nltk.tokenize import sent_tokenize
import numpy as np
import re
import pandas as pd
import string
from string import digits
from sklearn.utils import shuffle

lines = pd.read_csv('seq2seq_dataframe.csv')
lines = lines.head(1000)

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(lines.Query + lines.Prediction)
VOCAB_SIZE = len(tokenizer.word_index) + 1
print('VOCAB SIZE : {}'.format(VOCAB_SIZE))

questions = lines.Query
answers = lines.Prediction

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
onehot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)
print(decoder_output_data.shape)

shape = 100

encoder_inputs = layers.Input(shape=(None,))
encoder_embedding = layers.Embedding(VOCAB_SIZE, shape, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = layers.LSTM(shape, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = layers.Input(shape=(None,))
decoder_embedding = layers.Embedding(VOCAB_SIZE, shape, mask_zero=True)(decoder_inputs)
decoder_lstm = layers.LSTM(shape, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = layers.Dense(VOCAB_SIZE, activation=activations.softmax)
output = decoder_dense(decoder_outputs)

model = models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

model.fit([encoder_input_data, decoder_input_data], decoder_output_data, \
          batch_size=10, epochs=5)
model.save('model.h5')


def make_inference_models():
    encoder_model = models.Model(encoder_inputs, encoder_states)
    decoder_state_input_h = layers.Input(shape=(shape,))
    decoder_state_input_c = layers.Input(shape=(shape,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model, decoder_model


def str_to_tokens(sentence: str):
    words = sentence.lower().split()
    tokens_list = list()
    # return tokens_list
    for word in words:
        tokens_list.append(tokenizer.word_index[word])
    return preprocessing.sequence.pad_sequences([tokens_list], maxlen=maxlen_questions, padding='post')


enc_model, dec_model = make_inference_models()

for _ in range(10):
    try:
        states_values = enc_model.predict(str_to_tokens(input('Enter question : ')))
    except KeyError:
        continue
        pass
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                decoded_translation += ' {}'.format(word)
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        states_values = [h, c]

    print(decoded_translation)


VOCAB SIZE : 941
(1000, 5) 5
(1000, 7) 7
(1000, 7, 941)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    94100       input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    94100       input_6[0][0]                    
____________________________________

KeyError: 'setting'